<a href="https://colab.research.google.com/github/lgustavo95/CapacitaBR-CienDados/blob/main/QA_MANUAL2_5_COM_OVERLAP_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Instalação de Dependências
# Instala pacotes do sistema e bibliotecas Python necessárias para:
# - Extração de texto de PDFs (Tesseract OCR, pdfplumber, pymupdf)
# - Processamento de linguagem natural (transformers, sentence-transformers)
# - Visualização (ipywidgets, reportlab)
# - Gerenciamento de embeddings (chromadb)


!apt update # Atualiza lista de pacotes do sistema
!apt install -y tesseract-ocr tesseract-ocr-por # Instala Tesseract OCR com suporte a português
!pip install pymupdf transformers sentence-transformers torch pytesseract pillow nltk tqdm chromadb # Instala bibliotecas Python principais
!pip install pdfplumber reportlab ipywidgets # Instala bibliotecas adicionais para manipulação de PDFs e interface
!apt-get install -y ttf-mscorefonts-installer # Instala fontes Microsoft para melhor renderização de textos


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [4,572 kB]
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,036 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 P

In [ ]:
# @title Imports Necessários
# Bibliotecas padrão
import os # Para operações com sistema de arquivos
import warnings # Para gerenciamento de avisos
import pickle # Para serialização de objetos
import shutil # Para operações com arquivos/diretórios

# Bibliotecas para processamento de PDFs e imagen
import fitz  # PyMuPDF - usado para gerar imagem da página
import numpy as np
from PIL import Image # Para manipular imagem no Python
import pytesseract # OCR para extrair texto de imagem
import pdfplumber  # Para extrair texto diretamente de PDFs

# Bibliotecas para NLP (Processamento de Linguagem Natural)
import nltk  # Toolkit de NLP
from tqdm import tqdm # Para barras de progresso
from transformers import pipeline # Para modelos de NLP pré-treinados
from sentence_transformers import SentenceTransformer # Para embeddings de texto

# Bibliotecas para interface e visualização
from IPython.display import display, clear_output # Para exibição no Colab
import ipywidgets as widgets # Para criar widgets interativos
from google.colab import files # Para upload de arquivos no Colab

# Configurações iniciais
nltk.download('punkt') # Baixa dados necessários para tokenização
warnings.filterwarnings("ignore") # Ignora avisos para limpeza da saída
os.environ["TESSDATA_PREFIX"] = "/usr/share/tesseract-ocr/4.00/tessdata" # Caminho para dados do Tesseract


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# @title Extração de Texto do PDF (Versão Aprimorada)
# Permite ao usuário fazer upload de um arquivo PDF
# Nota: No Google Colab, isso abrirá uma interface para seleção de arquivo
uploaded = files.upload()

pdf_text = ""  # Variável para armazenar todo o texto extraído

for file_name in uploaded.keys():
    print(f"Processando arquivo: {file_name}...")

    try:
        # Tentativa 1: Extração com pdfplumber (melhor para PDFs textuais)
        with pdfplumber.open(file_name) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:  # Verifica se extraiu texto válido
                    pdf_text += page_text + "\n"

        # Se não extraiu texto suficiente, tenta com PyMuPDF
        if len(pdf_text.strip()) < 100:  # Limite arbitrário (ajuste conforme necessário)
            raise Exception("Texto insuficiente - tentando fallback com PyMuPDF")

    except Exception as e:
        print(f"pdfplumber não conseguiu extrair texto adequadamente ({str(e)}). Usando PyMuPDF como fallback...")

        # Tentativa 2: Fallback com PyMuPDF
        try:
            doc = fitz.open(file_name)
            for page in doc:
                page_text = page.get_text()
                if page_text:
                    pdf_text += page_text + "\n"

            # Se ainda não houver texto, tenta OCR com Tesseract (para PDFs de imagem)
            if len(pdf_text.strip()) < 100:
                print("PyMuPDF não extraiu texto suficiente. Tentando OCR...")
                for page in doc:
                    pix = page.get_pixmap()
                    img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
                    pdf_text += pytesseract.image_to_string(img, lang='por') + "\n"

        except Exception as e:
            print(f"Falha crítica ao processar o PDF: {str(e)}")

if pdf_text.strip():
    print("Texto extraído com sucesso!")
    print(f"Total de caracteres extraídos: {len(pdf_text)}")
else:
    print("AVISO: Não foi possível extrair texto significativo do PDF.")


Saving manual_gol-230-248.pdf to manual_gol-230-248 (1).pdf
Processando arquivo: manual_gol-230-248 (1).pdf...
Texto extraído com sucesso!
Total de caracteres extraídos: 50380


In [ ]:
# @title Divisão do texto em chunks com overlap (contexto preservado)
# Esta função divide o texto em blocos com sobreposição de contexto (overlap),
# ideal para embeddings, QA ou busca semântica com base vetorial (ChromaDB).

def split_text_with_overlap(texto, tamanho_chunk=600, overlap=150):
    """
    Divide o texto em blocos com overlap.

    Args:
        texto (str): Texto a dividir.
        tamanho_chunk (int): Número de caracteres por bloco.
        overlap (int): Número de caracteres que se repetem entre blocos consecutivos.

    Returns:
        list: Lista de chunks.
    """
    chunks = []
    start = 0
    while start < len(texto):
        end = start + tamanho_chunk
        chunk = texto[start:end]
        chunks.append(chunk.strip())
        start += tamanho_chunk - overlap
    return chunks

# Aplicar a divisão ao texto extraído (assumindo que pdf_text já existe)
chunks = split_text_with_overlap(pdf_text, tamanho_chunk=600, overlap=150)

# Exibir alguns chunks para revisão
for i, chunk in enumerate(chunks[:3]):
    print(f"Chunk {i+1} (tamanho: {len(chunk)}):\n{chunk}\n{'-'*60}")


Chunk 1 (tamanho: 600):
Rodas e pneus ● A eficiência dos sistemas de assistência ao
condutor e dos sistemas de assistência de
Informações importantes frenagem também depende da aderência dos
pneus.
sobre rodas e pneus
● Se, durante a condução, forem identificadas
vibrações estranhas ou o veículo estiver pu-
 Introdução ao tema xando para um dos lados, parar imediata-
mente e verificar as rodas e os pneus quanto
a danos.
A Volkswagen recomenda que todos os trabalhos ● Não utilizar rodas ou pneus de procedência
nas rodas e nos pneus sejam executados por uma desconhecida. Rodas e pneus usados podem
empresa especializad
------------------------------------------------------------
Chunk 2 (tamanho: 600):
ão utilizar rodas ou pneus de procedência
nas rodas e nos pneus sejam executados por uma desconhecida. Rodas e pneus usados podem
empresa especializada. Empresas especializadas estar danificados, mesmo se os danos não
estão equipadas com todas as ferramentas e pe- forem visíveis.
ças de re

In [ ]:
# @title Chunking e Embedding

def preprocess_text(text):
    import re
    text = text.replace('\n', ' ').replace('\r', ' ')
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def chunk_text(text, max_length=500):
    chunks = []
    start = 0
    while start < len(text):
        end = min(start + max_length, len(text))
        last_period = text.rfind('.', start, end)
        if last_period != -1 and last_period > start:
            end = last_period + 1
        chunk = text[start:end].strip()
        if chunk:
            chunks.append(chunk)
        start = end
    return chunks

processed_text = preprocess_text(pdf_text)
chunks = chunk_text(processed_text, max_length=500)

embed_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
embeddings = embed_model.encode(chunks, show_progress_bar=True)


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# @title Integração com ChromaDB
# @title Integração com ChromaDB
# ChromaDB é um banco de dados vetorial open-source para armazenar e pesquisar embeddings
import chromadb
from chromadb.config import Settings

if os.path.exists("/content/chroma_db"):
    shutil.rmtree("/content/chroma_db")

client = chromadb.Client()
collection = client.get_or_create_collection("manual_chunks")

for i, (chunk, embedding) in enumerate(zip(chunks, embeddings)):
    collection.add(documents=[chunk], embeddings=[embedding.tolist()], ids=[str(i)])


In [ ]:
# @title Caixa de Perguntas e Respostas
# Configura um pipeline de Question Answering usando um modelo BERT em português
# Modelo treinado para tarefas de resposta a perguntas no estilo SQuAD
qa_pipeline = pipeline(
    "question-answering",
    model="pierreguillou/bert-base-cased-squad-v1.1-portuguese",
    tokenizer="pierreguillou/bert-base-cased-squad-v1.1-portuguese"
)
# Função para responder perguntas:
# 1. Gera embedding da pergunta
# 2. Busca chunks similares no ChromaDB
# 3. Usa o modelo QA para encontrar respostas nos chunks
def responder_pergunta(pergunta, top_k=3):
    pergunta_embedding = embed_model.encode(pergunta)
    resultados = collection.query(query_embeddings=[pergunta_embedding.tolist()], n_results=top_k)
    respostas = []
    for doc in resultados['documents'][0]:
        resposta = qa_pipeline(question=pergunta, context=doc)
        respostas.append((resposta['answer'], resposta['score'], doc))
    return respostas

# Função chamada quando o botão "Responder" é clicado
def on_button_click(b):
    clear_output(wait=True)
    display(pergunta_box, button)
    pergunta = pergunta_box.value
    respostas = responder_pergunta(pergunta)
    for i, (resposta, score, contexto) in enumerate(respostas, 1):
        print(f"Resposta {i}:")
        print(f" - Texto: {contexto}")
        print(f" - Resposta: {resposta}")
        print(f" - Score: {score:.4f}\n")

pergunta_box = widgets.Text(value='', placeholder='Digite sua pergunta...', description='Pergunta:')
button = widgets.Button(description='Responder')
button.on_click(on_button_click)
display(pergunta_box, button)


Text(value='Qual o Torque de aperto dos parafusos da roda?', description='Pergunta:', placeholder='Digite sua …

Button(description='Responder', style=ButtonStyle())

Resposta 1:
 - Texto: pressão especificada no lado interno da portinho- la do tanque de combustível. Para soltar os parafusos da roda, utilizar somente A roda com aro de 14 polegadas deve retornar a chave de roda pertencente ao veículo. para a condição de roda de emergência o mais Enquanto o veículo não estiver levantado pelo breve possível, após a reinstalação da roda e macaco, soltar os parafusos da roda cerca de pneu normais do veículo, já reparados ou substi- uma volta apenas. tuídos.
 - Resposta: uma volta
 - Score: 0.0000

Resposta 2:
 - Texto: O torque de aperto especificado dos parafusos da ● Os parafusos da roda e os orifícios rosqueá- roda em aros de roda de aço e de liga leve é de veis dos cubos das rodas devem estar limpos, 120 Nm. Após uma troca de roda, o torque de de fácil manuseio e sem óleo e graxa. aperto deve ser verificado imediatamente com ● Utilizar apenas a chave de roda fornecida de um torquímetro calibrado.
 - Resposta: 120 Nm
 - Score: 0.4368

Resposta 3:
 - T